# Install necessary Libraries

In [ ]:
!pip install -U langchain-community
!pip install sentence_transformers
!pip install pinecone-client
!pip install pinecone-client[grpc]
!pip install nltk
!pip install beautifulsoup4

In [ ]:
import pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

api_key = "6ca3b069-c0d4-460e-8d64-9f545902b7c6"  
environment = "us-east-1" 

pc = Pinecone(api_key=api_key)

index_name = "chunk-embeddings-index"
dimension = 768  

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region=environment
        ) 
    )

index = pc.Index(index_name)

# Load Data, Chunk Legal Documents for Processing and setting up the vector database


In [ ]:
import numpy as np
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModel
import torch



tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz")
embedding_model = AutoModel.from_pretrained("camembert-base")
text_tokenizer = AutoTokenizer.from_pretrained("camembert-base")

def chunk_text_based_on_tokens(text, max_tokens=300):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(tokenizer.tokenize(sentence))
        if current_length + sentence_length <= max_tokens:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_length = sentence_length

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def extract_sections_articles_chapters(soup):
    sections = []
    current_section = []
    for element in soup.find_all(['h1', 'h2', 'h3', 'p']):
        if element.name in ['h1', 'h2', 'h3']:
            if current_section:
                sections.append(" ".join(current_section))
                current_section = []
            current_section.append(element.get_text())
        else:
            current_section.append(element.get_text())
    if current_section:
        sections.append(" ".join(current_section))
    return sections

def load_and_process_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = extract_sections_articles_chapters(soup)
    all_chunks = []
    for section in sections:
        all_chunks.extend(chunk_text_based_on_tokens(section))
    return all_chunks

def generate_bert_embedding(text, tokenizer, model, max_length=512):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,  
        padding=True,
        max_length=max_length 
    )
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze(0).numpy()
    return embedding.tolist()

def upsert_chunks_to_pinecone(index, namespace, chunks, batch_size=10):
    vectors = []
    for i, chunk in enumerate(chunks):
        embedding = generate_bert_embedding(chunk, text_tokenizer, embedding_model)
        vectors.append({
            "id": f"{namespace}_id_{i}", 
            "values": embedding,
            "metadata": {"text": chunk} 
        })
        
        if len(vectors) >= batch_size:
            index.upsert(vectors=vectors, namespace=namespace)
            vectors = []  
    
    if vectors:
        index.upsert(vectors=vectors, namespace=namespace)
        print(f"Upserted {len(vectors)} vectors to namespace '{namespace}'.")


def process_and_upload_html(file_path, namespace):
    print(f"Processing file: {file_path} for namespace: {namespace}")
    chunks = load_and_process_html(file_path)  
    upsert_chunks_to_pinecone(index, namespace, chunks) 

html_files_and_namespaces = {
    'gdpr': '/kaggle/input/datasetssss/french_gdpr.html',
    'ai_act': '/kaggle/input/datasetssss/french_AI_act.html',
    'dma': '/kaggle/input/datasetssss/french_dma.html',
    'dsa': '/kaggle/input/datasetssss/french_dsa.html'
}

for namespace, file_path in html_files_and_namespaces.items():
    process_and_upload_html(file_path, namespace)

In [ ]:
from sentence_transformers import SentenceTransformer

def embed_chunks(chunks, model_name):
    model = SentenceTransformer(model_name, device='cuda')
    
    embeddings = model.encode(chunks, normalize_embeddings=True)
    
    return embeddings

def embed_query(query, model_name):
    model = SentenceTransformer(model_name, device='cuda')
    query_embedding = model.encode([query], normalize_embeddings=True)
    return query_embedding[0]


def process_and_store_embeddings(file_path, namespace, model_name):
    chunks = load_and_process_html(file_path)
    embeddings = embed_chunks(chunks, model_name)

    vectors_to_upsert = [(f"id_{namespace}_{i}", embedding.tolist()) for i, embedding in enumerate(embeddings)]
    index.upsert(vectors=vectors_to_upsert, namespace=law)  

    return index, chunks, embeddings

def embed_query(query, model_name):
    model = SentenceTransformer(model_name, device='cpu')
    query_embedding = model.encode([query], normalize_embeddings=True)
    return query_embedding[0]


def query_pinecone_db(query_embedding, index, namespace, top_k=1):
    results = index.query(vector=query_embedding, top_k=top_k, namespace=law) 
    return results

# retrieve the most relevant chunk based on testing specific queries

In [ ]:
laws_info = {
    'gdpr': {
        'file_path': '/kaggle/input/datasetssss/french_gdpr.html',
        'index_name': 'index_name',
        'query': "Quels sont les objectifs principaux de l'harmonisation des règles de protection des données à caractère personnel au sein de l'Union Européenne selon ce texte?"
    },
    'ai_act': {
        'file_path': '/kaggle/input/datasetssss/french_AI_act.html',
        'index_name': 'index_name',
        'query': "Pourquoi la Commission européenne doit-elle évaluer et modifier chaque année la liste des systèmes d'IA à haut risque, et quel impact cela pourrait-il avoir sur le développement et la régulation des technologies d'IA ?"
    },
    'dma': {
        'file_path': '/kaggle/input/datasetssss/french_dma.html',
        'index_name': 'index_name',
        'query': "Quelle est l'importance de déléguer à la Commission européenne le pouvoir de modifier les annexes du règlement concernant les biens à double usage soumis à des contrôles d'exportation, et comment cela assure-t-il une réponse rapide de l'Union en cas d'évolution des circonstances?"
    },
    'dsa': {
        'file_path': '/kaggle/input/datasetssss/french_dsa.html',
        'index_name': 'index_name',
        'query': "Quel est l'objectif principal du règlement mentionné dans ce texte en relation avec les autres actes du droit de l'Union européenne?"
    }
}
model_name = ("camembert-base")

chunks_dict = {}
embeddings_dict = {}
import time

if index_name in pc.list_indexes().names():
    print(f"Deleting index '{index_name}'...")
    pc.delete_index(index_name)

    print("Waiting for the index to be deleted...")
    time.sleep(5)  

    if index_name not in pc.list_indexes().names():
        print(f"Index '{index_name}' successfully deleted.")
    else:
        print(f"Index '{index_name}' still exists, trying again...")
        pc.delete_index(index_name)
        time.sleep(5)  # Wait for another 5 seconds
        if index_name not in pc.list_indexes().names():
            print(f"Index '{index_name}' successfully deleted on second attempt.")
        else:
            print(f"Failed to delete index '{index_name}'. Please check your Pinecone dashboard.")
else:
    print(f"Index '{index_name}' does not exist or was already deleted.")

pc.create_index(
    name=index_name,
    dimension=dimension,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws', 
        region=environment
    )
)
print(f"Index '{index_name}' created successfully.")

for law, info in laws_info.items():
    print(f"Processing {law}...")
    index, chunks, embeddings = process_and_store_embeddings(info['file_path'], info['index_name'], model_name)
    chunks_dict[law] = chunks  
    embeddings_dict[law] = embeddings

    vectors_to_upsert = [(f"{law}_id_{i}", embedding.tolist()) for i, embedding in enumerate(embeddings)]
    
    try:
        print(f"Clearing vectors for namespace '{law}'...")
        index.delete(delete_all=True, namespace=law)  
        print(f"Namespace '{law}' cleared.")
    except Exception as e:
        print(f"Namespace '{law}' does not exist or an error occurred: {e}")
    
    try:
        print(f"Upserting vectors for namespace '{law}'...")
        index.upsert(vectors=vectors_to_upsert, namespace=law)
        print(f"Upsert complete for namespace '{law}'.")
    except Exception as e:
        print(f"Failed to upsert vectors for namespace '{law}': {e}")

print("Chunks Dict Content: ")
for key in chunks_dict.keys():
    print(f"Law: {key}, Number of Chunks: {len(chunks_dict[key])}")



In [ ]:
# Query each law and print results
for law, info in laws_info.items():
    print(f"\nQuerying {law.upper()} index:")
    query_embedding = embed_query(info['query'], model_name)
    print(info['query'])
    results = query_pinecone_db(query_embedding, index, namespace=law, top_k=1)  
    
    if results and 'matches' in results and results['matches']:
        retrieved_chunk_id = results['matches'][0]['id']
        retrieved_chunk = chunks_dict[law][int(retrieved_chunk_id.split('_')[-1])]
        print(f"Retrieved chunk {retrieved_chunk_id.split('_')[-1]} from {law.upper()}:")
        print(retrieved_chunk)
    else:
        print(f"No results found for {law.upper()}.")

# measuring the relevancy of the retrieved texts and the answers

In [ ]:
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util

tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModel.from_pretrained("camembert-base")

semantic_model = SentenceTransformer('camembert-base', device='cuda')

def generate_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].numpy() 
    return embedding

def calculate_cosine_similarity(reference_embedding, retrieved_embedding):
    return cosine_similarity(reference_embedding.reshape(1, -1), retrieved_embedding.reshape(1, -1))[0][0]

def calculate_semantic_similarity(reference_text, retrieved_text, model):
    embeddings1 = model.encode(reference_text, convert_to_tensor=True)
    embeddings2 = model.encode(retrieved_text, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embeddings1, embeddings2)
    return similarity.item()

reference_answers = {
    'gdpr': "Afin d'assurer un niveau cohérent et élevé de protection des personnes physiques et de lever les obstacles aux flux de données à caractère personnel au sein de l'Union, le niveau de protection des droits et des libertés des personnes physiques à l'égard du traitement de ces données devrait être équivalent dans tous les États membres. Il convient dès lors d'assurer une application cohérente et homogène des règles de protection des libertés et droits fondamentaux des personnes physiques à l'égard du traitement des données à caractère personnel dans l'ensemble de l'Union. En ce qui concerne le traitement de données à caractère personnel nécessaire au respect d'une obligation légale, à l'exécution d'une mission d'intérêt public ou relevant de l'exercice de l'autorité publique dont est investi le responsable du traitement, il y a lieu d'autoriser les États membres à maintenir ou à introduire des dispositions nationales destinées à préciser davantage l'application des règles du présent règlement. Parallèlement à la législation générale et horizontale relative à la protection des données mettant en œuvre la directive 95/46/CE, il existe, dans les États membres, plusieurs législations sectorielles spécifiques dans des domaines qui requièrent des dispositions plus précises. Le présent règlement laisse aussi aux États membres une marge de manœuvre pour préciser ses règles, y compris en ce qui concerne le traitement de catégories particulières de données à caractère personnel (ci-après dénommées «données sensibles»). À cet égard, le présent règlement n'exclut pas que le droit des États membres précise les circonstances des situations particulières de traitement y compris en fixant de manière plus précise les conditions dans lesquelles le traitement de données à caractère personnel est licite.",
    'ai_act': "Compte tenu de l’évolution rapide des technologies et de l’expertise technique requise aux fins de la bonne application du présent règlement, la Commission devrait évaluer et réexaminer le présent règlement au plus tard le 2 août 2029 et tous les quatre ans par la suite, et faire rapport au Parlement européen et au Conseil. En outre, en tenant compte des conséquences sur le champ d’application du présent règlement, la Commission devrait procéder à une évaluation de la nécessité de modifier une fois par an la liste des systèmes d’IA à haut risque et la liste des pratiques interdites. En outre, au plus tard le 2 août 2028 et tous les quatre ans par la suite, la Commission devrait évaluer la nécessité de modifier les rubriques de la liste des domaines à haut risque figurant à l’annexe du présent règlement, les systèmes d’IA relevant des obligations de transparence, l’efficacité du système de surveillance et de gouvernance ainsi que l’état d’avancement des travaux de normalisation concernant le développement économe en énergie de modèles d’IA à usage général, y compris la nécessité de mesures ou d’actions supplémentaires, et faire rapport au Parlement européen et au Conseil. Enfin, au plus tard le 2 août 2028 et tous les trois ans par la suite, la Commission devrait évaluer l’impact et l’efficacité des codes de conduite volontaires destinés à favoriser l’application des exigences énoncées pour les systèmes d’IA à haut risque dans le cas des systèmes d’IA autres que les systèmes d’IA à haut risque, et éventuellement d’autres exigences supplémentaires pour de tels systèmes d’IA.",
    'dma': "Afin de permettre à l’Union d’agir rapidement en cas d’évolution des circonstances prises en compte pour apprécier le caractère sensible d’exportations au titre d’autorisations générales d’exportation de l’Union, ainsi que des progrès technologiques ou de l’évolution de la conjoncture commerciale, il convient de déléguer à la Commission le pouvoir d’adopter des actes conformément à l’article 290 du traité sur le fonctionnement de l’Union européenne pour la modification des annexes I, II et IV du présent règlement. Les décisions relatives à la mise à jour de la liste commune des biens à double usage soumis à des contrôles d’exportation figurant à l’annexe I devraient être conformes aux obligations et engagements que les États membres ou l’Union ont acceptés en tant que membres des accords internationaux pertinents de prolifération et en tant que membres des régimes multilatéraux pertinents de contrôle des exportations, ou du fait de la ratification de traités internationaux en la matière. Lorsque la modification de l’annexe I porte sur les biens à double usage qui sont également énumérés à l’annexe II ou IV, ces annexes devraient également être modifiées en conséquence. Les décisions relatives à la mise à jour de la liste commune des biens à double usage soumis à des contrôles d’exportation figurant à l’annexe IV devraient être prises en tenant compte des intérêts d’ordre public et de sécurité publique des États membres, en vertu de l’article 36 du traité sur le fonctionnement de l’Union européenne. Les décisions relatives à la mise à jour des listes communes des biens et destinations figurant à l’annexe II, sections A à H, devraient être prises en tenant compte des critères d’appréciation énoncés dans le présent règlement. Il importe particulièrement que la Commission procède aux consultations appropriées durant son travail préparatoire, y compris au niveau des experts, et que ces consultations soient menées conformément aux principes définis dans l’accord interinstitutionnel du 13 avril 2016«Mieux légiférer» (5). En particulier, pour assurer leur égale participation à la préparation des actes délégués, le Parlement européen et le Conseil reçoivent tous les documents au même moment que les experts des États membres, et leurs experts ont systématiquement accès aux réunions des groupes d’experts de la Commission traitant de la préparation des actes délégués.",
    'dsa': "Il convient que le présent règlement soit sans préjudice d’autres actes du droit de l’Union régissant la fourniture de services de la société de l’information en général, régissant d’autres aspects de la fourniture de services intermédiaires dans le marché intérieur ou précisant et complétant les règles harmonisées énoncées dans le présent règlement, tels que la directive 2010/13/UE du Parlement européen et du Conseil (7), y compris les dispositions de ladite directive concernant les plateformes de partage de vidéos, les règlements (UE) 2019/1148 (8), (UE) 2019/1150 (9), (UE) 2021/784 (10) et (UE) 2021/1232 (11) du Parlement européen et du Conseil et la directive 2002/58/CE du Parlement européen et du Conseil (12) et les dispositions du droit de l’Union énoncées dans un règlement relatif aux injonctions européennes de production et de conservation de preuves électroniques en matière pénale et dans une directive établissant des règles harmonisées concernant la désignation de représentants légaux aux fins de la collecte de preuves en matière pénale.De même, par souci de clarté, le présent règlement devrait être sans préjudice du droit de l’Union en matière de protection des consommateurs, en particulier les règlements (UE) 2017/2394 (13) et (UE) 2019/1020 (14) du Parlement européen et du Conseil, les directives 2001/95/CE (15), 2005/29/CE (16), 2011/83/UE (17) et 2013/11/UE (18) du Parlement européen et du Conseil et la directive 93/13/CEE du Conseil (19), et en matière de protection des données à caractère personnel, en particulier le règlement (UE) 2016/679 du Parlement européen et du Conseil (20).Il convient également que le présent règlement soit sans préjudice des règles de l’Union en matière de droit international privé, en particulier les règles relatives à la compétence ainsi qu’à la reconnaissance et à l’exécution des décisions en matière civile et commerciale, comme le règlement (UE) n° 1215/2012, et les règles relatives à la loi applicable aux obligations contractuelles et non contractuelles. La protection des personnes au regard du traitement des données à caractère personnel est régie exclusivement par les règles du droit de l’Union en la matière, en particulier le règlement (UE) 2016/679 et la directive 2002/58/CE. Il convient également que le présent règlement soit sans préjudice du droit de l’Union relatif aux conditions de travail et du droit de l’Union dans le domaine de la coopération judiciaire en matière civile et pénale. Toutefois, dans la mesure où ces actes juridiques de l’Union poursuivent les mêmes objectifs que ceux énoncés dans le présent règlement, les règles du présent règlement devraient s’appliquer en ce qui concerne les aspects qui ne sont pas ou ne sont pas pleinement traités par ces autres actes juridiques ainsi que les aspects pour lesquels ces autres actes juridiques laissent aux États membres la possibilité d’adopter certaines mesures au niveau national."
}

In [ ]:
similarities = []

for law, info in laws_info.items():
    print(f"\nQuerying {law.upper()} index:")
    query_embedding = embed_query(info['query'], model_name)
    print(info['query'])
    results = query_pinecone_db(query_embedding, index, namespace=law, top_k=1)  
    
    if results and 'matches' in results and results['matches']:
        retrieved_chunk_id = results['matches'][0]['id']
        retrieved_chunk = chunks_dict[law][int(retrieved_chunk_id.split('_')[-1])]
        print(f"Retrieved chunk {retrieved_chunk_id.split('_')[-1]} from {law.upper()}:")
        print(retrieved_chunk)

        retrieved_embedding = generate_bert_embedding(retrieved_chunk, tokenizer, model)
        reference_embedding = generate_bert_embedding(reference_answers[law], tokenizer, model)
        cosine_sim = calculate_cosine_similarity(reference_embedding, retrieved_embedding)
        
        semantic_sim = calculate_semantic_similarity(reference_answers[law], retrieved_chunk, semantic_model)

        similarities.append({
            'law': law,
            'retrieved_answer': retrieved_chunk,
            'cosine_similarity': cosine_sim,
            'semantic_similarity': semantic_sim
        })

        print(f"Cosine Similarity with reference answer: {cosine_sim:.4f}")
        print(f"Semantic Similarity with reference answer: {semantic_sim:.4f}")
        print("----\n")
    else:
        print(f"No results found for {law.upper()} in the query.")

# setting 20 questions and their answers for each law, make embedidngs of them while making summarizations of the most relevant chunks that have beed retrieved.

In [ ]:
integrated_questions_answers = [
    # Question 1 from GDPR
    {
        'law': 'gdpr',
        'question': "Quel est le droit fondamental concernant le traitement des données personnelles selon la Charte des droits fondamentaux de l'Union européenne?",
        'answer': "La protection des personnes physiques à l'égard du traitement des données personnelles est un droit fondamental. L'article 8(1) de la Charte des droits fondamentaux de l'Union européenne (« la Charte ») et l'article 16(1) du Traité sur le fonctionnement de l'Union européenne (TFUE) prévoient que toute personne a droit à la protection des données personnelles la concernant. Ce règlement vise à contribuer à la réalisation d'un espace de liberté, de sécurité et de justice et d'une union économique, au progrès économique et social, au renforcement et à la convergence des économies au sein du marché intérieur, ainsi qu'au bien-être des personnes physiques."
    },
    # Question 1 from AI Act
    {
        'law': 'ai_act',
        'question': "Quels sont les principaux objectifs de l'AI Act concernant le développement et l'utilisation de l'IA dans l'Union européenne?",
        'answer': "L'AI Act vise à garantir que les systèmes d'IA mis sur le marché et utilisés dans l'Union sont sûrs, respectent les lois existantes sur les droits fondamentaux et les valeurs de l'Union, et ne compromettent pas les droits fondamentaux. La loi vise à établir un cadre juridique qui traite les risques posés par l'IA, en particulier les systèmes d'IA à haut risque, et à améliorer la transparence, la responsabilité et la confiance dans l'IA tout en promouvant l'innovation et la compétitivité."
    },
    # Question 1 from DMA
    {
        'law': 'dma',
        'question': "Quels critères sont utilisés pour définir un 'contrôleur d'accès' selon la Loi sur les marchés numériques?",
        'answer': "Un contrôleur d'accès selon la DMA est défini comme un fournisseur de services de plateforme essentiels qui a un impact significatif sur le marché intérieur, sert de porte d'accès importante pour les utilisateurs professionnels pour atteindre les utilisateurs finaux, et jouit d'une position bien établie et durable sur le marché. Les critères incluent une position économique forte, un grand nombre d'utilisateurs, et le contrôle d'un écosystème difficile à contester pour les autres entreprises."
    },
    # Question 1 from DSA
    {
        'law': 'dsa',
        'question': "Quelles sont les principales responsabilités des plateformes en ligne selon la Loi sur les services numériques?",
        'answer': "Selon la DSA, les plateformes en ligne sont responsables de prendre des mesures efficaces pour atténuer les risques liés aux contenus illégaux, assurer la sécurité des utilisateurs et protéger les droits fondamentaux. Les plateformes doivent mettre en place des mécanismes de signalement et de suppression des contenus illégaux, fournir aux utilisateurs des conditions générales claires, et établir des processus de traitement des plaintes et des appels. Les plateformes qui atteignent un nombre significatif d'utilisateurs doivent également évaluer et atténuer les risques systémiques, tels que la propagation de la désinformation et des contenus nuisibles."
    },
    # Question 2 from GDPR
    {
        'law': 'gdpr',
        'question': "Comment le RGPD vise-t-il à équilibrer le droit à la protection des données personnelles avec d'autres droits fondamentaux?",
        'answer': "Ce règlement respecte tous les droits fondamentaux et observe les libertés et principes reconnus dans la Charte tels qu'inscrits dans les traités, notamment le respect de la vie privée et familiale, du domicile et des communications, la protection des données personnelles, la liberté de pensée, de conscience et de religion, la liberté d'expression et d'information, la liberté d'entreprendre, le droit à un recours effectif et à un procès équitable, et la diversité culturelle, religieuse et linguistique. Le droit à la protection des données personnelles doit être considéré par rapport à sa fonction dans la société et être mis en balance avec d'autres droits fondamentaux, conformément au principe de proportionnalité."
    },
    # Question 2 from AI Act
    {
        'law': 'ai_act',
        'question': "Comment l'AI Act propose-t-il de réglementer les systèmes d'IA à haut risque?",
        'answer': "L'AI Act classe les systèmes d'IA en fonction du risque qu'ils posent et soumet les systèmes d'IA à haut risque à des exigences strictes. Les systèmes d'IA à haut risque incluent ceux utilisés dans les infrastructures critiques, l'éducation, l'emploi, les services publics et privés essentiels, l'application de la loi, et la gestion de la migration, de l'asile et du contrôle des frontières. Ces systèmes doivent se conformer à des exigences liées à la gestion des risques, à la gouvernance des données, à la documentation technique, à la tenue de registres, à la transparence, à la fourniture d'informations aux utilisateurs, à la surveillance humaine, à l'exactitude et à la robustesse. Les fournisseurs de ces systèmes doivent établir un système de gestion de la qualité et assurer une surveillance continue et une surveillance post-marché."
    },
    # Question 2 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA propose-t-elle de réglementer le comportement des contrôleurs d'accès dans les marchés numériques?",
        'answer': "La DMA impose des obligations spécifiques aux contrôleurs d'accès pour les empêcher de se livrer à des pratiques déloyales qui nuisent à la concurrence et aux consommateurs. Cela inclut l'interdiction pour les contrôleurs d'accès de favoriser leurs propres services par rapport à ceux des concurrents (auto-préférence), l'obligation de permettre l'interopérabilité avec les services tiers, et de s'assurer qu'ils ne limitent pas injustement l'accès à leurs plateformes. Les contrôleurs d'accès sont également tenus de fournir la portabilité des données, d'offrir des conditions équitables aux utilisateurs professionnels et d'assurer la transparence de leurs opérations."
    },
    # Question 2 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA vise-t-elle à protéger les utilisateurs contre les contenus illégaux sur les plateformes numériques?",
        'answer': "La DSA vise à protéger les utilisateurs contre les contenus illégaux en exigeant que les plateformes mettent en œuvre des mécanismes de notification et d'action, permettant aux utilisateurs de signaler facilement les contenus illégaux. Les plateformes doivent agir rapidement pour supprimer ou désactiver l'accès aux contenus illégaux dès réception d'une notification. La DSA introduit également des obligations pour les plateformes de coopérer avec les forces de l'ordre et de fournir des rapports de transparence sur leurs activités de modération de contenu. Les plateformes doivent prendre des mesures proactives pour prévenir la propagation de contenus illégaux et veiller à ce que leurs algorithmes ne favorisent pas des contenus nuisibles ou illégaux."
    },
    # Question 3 from GDPR
    {
        'law': 'gdpr',
        'question': "Quels défis ont émergé en raison des développements technologiques et de la mondialisation dans le contexte de la protection des données personnelles?",
        'answer': "Les développements technologiques et la mondialisation ont apporté de nouveaux défis pour la protection des données personnelles. L'ampleur de la collecte et du partage des données personnelles a considérablement augmenté. La technologie permet aux entreprises privées et aux autorités publiques d'utiliser les données personnelles à une échelle sans précédent pour mener leurs activités. Les personnes physiques rendent de plus en plus leurs informations personnelles disponibles publiquement et mondialement. La technologie a transformé à la fois l'économie et la vie sociale, et devrait encore faciliter le libre flux des données personnelles au sein de l'Union et le transfert vers des pays tiers et des organisations internationales, tout en garantissant un niveau élevé de protection des données personnelles."
    },
    # Question 3 from AI Act
    {
        'law': 'ai_act',
        'question': "Quelles responsabilités l'AI Act impose-t-il aux fournisseurs d'IA pour garantir des pratiques éthiques d'IA?",
        'answer': "Les fournisseurs de systèmes d'IA à haut risque sont responsables de garantir que leurs systèmes sont conformes aux exigences énoncées dans la loi. Cela inclut l'obligation de réaliser une évaluation de conformité avant de mettre le système sur le marché, de s'assurer que le système subit des tests appropriés, de fournir des instructions claires et des informations aux utilisateurs, de mettre en œuvre des mesures de surveillance humaine, et de surveiller le système tout au long de son cycle de vie. Les fournisseurs doivent également signaler aux autorités les incidents graves et les dysfonctionnements."
    },
    # Question 3 from DMA
    {
        'law': 'dma',
        'question': "Quelles sont les principales obligations imposées aux contrôleurs d'accès par la DMA?",
        'answer': "Les principales obligations pour les contrôleurs d'accès en vertu de la DMA incluent l'interdiction de combiner des données personnelles provenant de différentes sources sans le consentement de l'utilisateur, des restrictions sur l'installation de logiciels ou d'applications préinstallés, et l'exigence de permettre aux utilisateurs professionnels d'accéder aux données générées sur leur plateforme. Les contrôleurs d'accès doivent également s'assurer que leurs plateformes sont ouvertes et interopérables avec les services tiers, et ils sont interdits d'utiliser des données non publiques de leurs utilisateurs professionnels pour concurrencer ces derniers."
    },
    # Question 3 from DSA
    {
        'law': 'dsa',
        'question': "Quelles exigences de transparence sont imposées aux plateformes en ligne par la DSA?",
        'answer': "La DSA impose des exigences de transparence étendues aux plateformes en ligne, y compris l'obligation de publier des rapports de transparence détaillant le nombre d'actions de suppression de contenu, les raisons de ces actions, et les résultats des appels des utilisateurs. Les plateformes doivent également divulguer comment fonctionnent leurs systèmes de modération de contenu et leurs algorithmes de recommandation, y compris les critères utilisés pour classer et afficher le contenu. Les utilisateurs doivent être informés des conditions d'utilisation de la plateforme et des modifications apportées à ces conditions. De plus, les plateformes doivent fournir des informations claires sur la publicité qu'elles diffusent, y compris l'identité des annonceurs et les critères de ciblage utilisés."
    },
    # Question 4 from GDPR
    {
        'law': 'gdpr',
        'question': "Comment le RGPD aborde-t-il le transfert de données personnelles vers des pays tiers ou des organisations internationales?",
        'answer': "Le transfert de données personnelles vers des pays tiers ou des organisations internationales n'est autorisé que si les conditions énoncées dans ce règlement sont respectées, afin de garantir que le niveau de protection des personnes physiques assuré par ce règlement ne soit pas compromis. En tout état de cause, les transferts vers des pays tiers et des organisations internationales ne peuvent être effectués que dans le plein respect de ce règlement. Ce règlement est sans préjudice des accords internationaux conclus entre l'Union et des pays tiers régissant le transfert de données personnelles, y compris des garanties appropriées pour les personnes concernées."
    },
    # Question 4 from AI Act
    {
        'law': 'ai_act',
        'question': "Comment l'AI Act aborde-t-il la transparence et la responsabilité des systèmes d'IA?",
        'answer': "L'AI Act exige que les systèmes d'IA, en particulier ceux à haut risque, soient transparents et fournissent des informations claires sur leur objectif, leurs capacités et leurs limites. Les utilisateurs doivent pouvoir comprendre comment les décisions sont prises par les systèmes d'IA et quelles données sont traitées. La loi exige que les systèmes d'IA soient conçus avec des fonctionnalités qui garantissent la responsabilité, y compris la capacité d'audit, la traçabilité des décisions, et la capacité de fournir des explications pour les décisions prises par l'IA."
    },
    # Question 4 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA vise-t-elle à prévenir les pratiques déloyales sur le marché numérique?",
        'answer': "La DMA vise à prévenir les pratiques déloyales en établissant des règles claires pour les contrôleurs d'accès, y compris l'interdiction de l'auto-préférence, des restrictions sur les termes et conditions injustes pour les utilisateurs professionnels, et des exigences de transparence sur leur fonctionnement. La DMA garantit également que les contrôleurs d'accès ne peuvent pas utiliser leur position dominante pour étouffer la concurrence ou l'innovation par les petites entreprises. La Commission européenne est habilitée à enquêter et à sanctionner les contrôleurs d'accès qui ne respectent pas ces règles."
    },
    # Question 4 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA propose-t-elle de gérer la diffusion de contenus nuisibles?",
        'answer': "La DSA propose de gérer la diffusion de contenus nuisibles en exigeant que les plateformes évaluent les risques associés à la diffusion de contenus nuisibles ou illégaux et prennent des mesures appropriées pour atténuer ces risques. Les plateformes doivent mettre en place des garde-fous pour s'assurer que leurs algorithmes ne favorisent pas les contenus nuisibles, et elles doivent fournir aux utilisateurs des outils pour contrôler les contenus auxquels ils sont exposés. La DSA encourage également les plateformes à coopérer avec des signaleurs de confiance et des vérificateurs de faits pour identifier et traiter plus efficacement les contenus nuisibles. Dans les cas où les plateformes ne parviennent pas à atténuer les risques de manière adéquate, elles peuvent faire l'objet de mesures réglementaires, y compris des amendes et d'autres sanctions."
    },
    # Question 5 from GDPR
    {
        'law': 'gdpr',
        'question': "Quelles protections spécifiques le RGPD offre-t-il aux enfants concernant leurs données personnelles?",
        'answer': "Les enfants méritent une protection spécifique en ce qui concerne leurs données personnelles, car ils peuvent être moins conscients des risques, des conséquences, des garanties et des droits liés au traitement des données personnelles. Cette protection spécifique devrait s'appliquer, en particulier, à l'utilisation des données personnelles des enfants à des fins de marketing ou de création de profils de personnalité ou d'utilisateur, ainsi qu'à la collecte de données personnelles concernant les enfants lors de l'utilisation de services proposés directement à un enfant. Le consentement du titulaire de la responsabilité parentale ne devrait pas être nécessaire dans le cadre de services de prévention ou de conseil offerts directement à un enfant."
    },
    # Question 5 from AI Act
    {
        'law': 'ai_act',
        'question': "Quelles mesures sont suggérées par l'AI Act pour protéger les droits fondamentaux dans le déploiement des technologies d'IA?",
        'answer': "L'AI Act intègre plusieurs mesures pour protéger les droits fondamentaux, telles que l'exigence que les systèmes d'IA soient conçus et utilisés d'une manière conforme au respect de la dignité humaine, de la vie privée, de la non-discrimination et d'autres droits fondamentaux. Cela inclut l'intégration de mécanismes de surveillance humaine, la garantie que les systèmes d'IA ne conduisent pas à des résultats biaisés ou discriminatoires, et la fourniture de voies pour que les individus contestent les décisions prises par les systèmes d'IA qui les affectent de manière significative. La loi encourage également le développement de codes de conduite et de mesures volontaires par les fournisseurs pour s'assurer que l'IA est utilisée de manière éthique et en alignement avec les valeurs sociétales."
    },
    # Question 5 from DMA
    {
        'law': 'dma',
        'question': "Quels mécanismes de mise en application sont inclus dans la DMA pour garantir la conformité des contrôleurs d'accès?",
        'answer': "La DMA comprend des mécanismes de mise en application robustes, tels que la possibilité pour la Commission européenne d'imposer des amendes allant jusqu'à 10 % du chiffre d'affaires annuel mondial total du contrôleur d'accès pour non-conformité. En cas d'infractions répétées, la Commission peut imposer des sanctions supplémentaires, y compris des mesures structurelles, telles que la cession d'entreprises. La DMA permet également des paiements périodiques de pénalités pour s'assurer que les contrôleurs d'accès respectent les obligations et interdictions énoncées dans la réglementation."
    },
    # Question 5 from DSA
    {
        'law': 'dsa',
        'question': "Quelles mesures la DSA inclut-elle pour protéger la liberté d'expression tout en combattant les contenus illégaux?",
        'answer': "La DSA inclut des mesures pour protéger la liberté d'expression en s'assurant que toute restriction de contenu est nécessaire, proportionnée et légalement justifiée. Les plateformes doivent fournir aux utilisateurs des explications claires lorsque du contenu est supprimé ou que l'accès est restreint, et les utilisateurs doivent avoir le droit de faire appel de ces décisions. La DSA exige également que les plateformes s'assurent que les processus de modération de contenu sont équitables et transparents, avec des garde-fous en place pour prévenir la suppression arbitraire de contenu. De plus, la DSA encourage les plateformes à développer des codes de conduite en collaboration avec les parties prenantes pour équilibrer la nécessité de lutter contre les contenus illégaux avec la protection de la liberté d'expression."
    },
    # Question 6 from GDPR
    {
        'law': 'gdpr',
        'question': "Comment le RGPD définit-il les données personnelles et quels sont quelques exemples?",
        'answer': "Les données personnelles selon le RGPD sont définies comme toute information relative à une personne physique identifiée ou identifiable (« personne concernée »). Les exemples incluent le nom d'une personne, son numéro d'identification, des données de localisation, un identifiant en ligne ou un ou plusieurs facteurs spécifiques à l'identité physique, physiologique, génétique, mentale, économique, culturelle ou sociale de cette personne physique. La définition est large, capturant diverses formes de données qui pourraient être utilisées pour identifier directement ou indirectement un individu."
    },
    # Question 6 from AI Act
    {
        'law': 'ai_act',
        'question': "Quelles catégories de systèmes d'IA sont considérées comme à haut risque selon l'AI Act?",
        'answer': "Les systèmes d'IA à haut risque selon l'AI Act incluent ceux utilisés dans les infrastructures critiques (telles que le transport, l'énergie et l'approvisionnement en eau), la formation éducative et professionnelle, l'emploi et la gestion des travailleurs, l'accès aux services privés et publics essentiels (tels que le scoring de crédit et les prestations sociales), l'application de la loi (telle que la police prédictive), la gestion de la migration, de l'asile et du contrôle des frontières, et l'administration de la justice et des processus démocratiques. Ces systèmes sont soumis à des exigences strictes en raison des risques importants qu'ils posent pour les droits fondamentaux et la sécurité."
    },
    # Question 6 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA aborde-t-elle la question de l'auto-préférence par les contrôleurs d'accès?",
        'answer': "La DMA interdit spécifiquement aux contrôleurs d'accès de se livrer à des pratiques d'auto-préférence, où ils favorisent leurs propres produits ou services par rapport à ceux des concurrents sur leurs plateformes. Cela inclut des pratiques telles que le classement de leurs propres produits plus haut dans les résultats de recherche ou l'octroi d'un accès préférentiel aux données. L'objectif est de garantir des conditions équitables sur les marchés numériques, où la concurrence est basée sur le mérite plutôt que sur le pouvoir de marché du contrôleur d'accès. L'interdiction de l'auto-préférence est l'une des principales obligations imposées aux contrôleurs d'accès pour prévenir les comportements anticoncurrentiels."
    },
    # Question 6 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA aborde-t-elle la question de la modération de contenu sur les plateformes en ligne?",
        'answer': "La DSA exige que les plateformes en ligne mettent en place des politiques de modération de contenu qui soient transparentes, cohérentes et alignées avec les droits fondamentaux. Les plateformes doivent établir des conditions générales claires pour la modération du contenu et fournir aux utilisateurs des informations détaillées sur la manière dont le contenu est évalué, supprimé ou restreint. La DSA impose également que les plateformes mettent en œuvre des mécanismes permettant aux utilisateurs de faire appel des décisions de modération de contenu, garantissant que les utilisateurs ont la possibilité de contester les suppressions ou restrictions injustifiées. Ces mesures visent à créer un système de modération de contenu équitable et responsable qui respecte la liberté d'expression tout en combattant les contenus illégaux."
    },
    # Question 7 from GDPR
    {
        'law': 'gdpr',
        'question': "Quelle est la base juridique du traitement des données personnelles selon le RGPD?",
        'answer': "Le RGPD énonce plusieurs bases juridiques pour le traitement des données personnelles, notamment : le consentement de la personne concernée ; le traitement nécessaire à l'exécution d'un contrat auquel la personne concernée est partie ; le traitement nécessaire au respect d'une obligation légale ; le traitement nécessaire à la protection des intérêts vitaux de la personne concernée ou d'une autre personne physique ; le traitement nécessaire à l'exécution d'une mission d'intérêt public ou à l'exercice de l'autorité publique ; et le traitement nécessaire aux fins des intérêts légitimes poursuivis par le responsable du traitement ou un tiers, sauf lorsque ces intérêts sont supplantés par les intérêts ou les droits et libertés fondamentaux de la personne concernée."
    },
    # Question 7 from AI Act
    {
        'law': 'ai_act',
        'question': "Comment l'AI Act définit-il 'système d'IA' et quelles technologies relèvent de cette définition?",
        'answer': "L'AI Act définit un 'système d'IA' comme un logiciel développé avec une ou plusieurs des techniques et approches listées dans la loi, telles que l'apprentissage automatique, les approches logiques et basées sur les connaissances, et les approches statistiques. Ces systèmes peuvent, pour un ensemble d'objectifs définis par l'humain, générer des résultats tels que du contenu, des prévisions, des recommandations ou des décisions influençant les environnements avec lesquels ils interagissent. La définition est large et inclut une variété de technologies d'IA, allant des algorithmes simples aux modèles d'apprentissage automatique complexes."
    },
    # Question 7 from DMA
    {
        'law': 'dma',
        'question': "Quels sont les critères pour identifier les services de plateforme essentiels selon la DMA?",
        'answer': "Les services de plateforme essentiels selon la DMA incluent une gamme de services numériques qui servent de passerelles importantes pour que les utilisateurs professionnels atteignent les utilisateurs finaux. Ces services comprennent les services d'intermédiation en ligne, tels que les magasins d'applications et les places de marché, les moteurs de recherche en ligne, les services de réseaux sociaux, les services de plateformes de partage de vidéos, les services de communication interpersonnelle indépendants du numéro, les systèmes d'exploitation, les services de cloud computing et les services publicitaires. Un service est considéré comme un service de plateforme essentiel s'il a un impact significatif sur le marché intérieur et constitue une passerelle essentielle pour que les utilisateurs professionnels accèdent aux utilisateurs finaux."
    },
    # Question 7 from DSA
    {
        'law': 'dsa',
        'question': "Quelles obligations ont les très grandes plateformes en ligne (VLOPs) selon la DSA?",
        'answer': "Les VLOPs, définies comme des plateformes comptant plus de 45 millions d'utilisateurs dans l'UE, ont des obligations supplémentaires selon la DSA en raison de leur impact significatif sur la société et le discours public. Les VLOPs doivent effectuer des évaluations annuelles des risques pour identifier et atténuer les risques systémiques, tels que la diffusion de contenus illégaux, la désinformation et les contenus nuisibles. Elles sont également tenues de fournir une plus grande transparence sur leurs algorithmes de recommandation de contenu, d'offrir aux utilisateurs plus de contrôle sur les contenus qu'ils voient, et de coopérer avec les autorités pour prévenir et traiter les risques systémiques. Ces obligations visent à garantir que les VLOPs opèrent de manière sûre, transparente et respectueuse des droits fondamentaux."
    },
    # Question 8 from GDPR
    {
        'law': 'gdpr',
        'question': "Quels sont les droits des personnes concernées par le RGPD?",
        'answer': "Le RGPD accorde aux personnes concernées plusieurs droits, notamment le droit d'être informé, le droit d'accès, le droit de rectification, le droit à l'effacement (« droit à l'oubli »), le droit de restreindre le traitement, le droit à la portabilité des données, le droit de s'opposer au traitement, et les droits relatifs à la prise de décision automatisée et au profilage. Ces droits permettent aux individus de contrôler leurs données personnelles et assurent la transparence et la responsabilité dans le traitement des données."
    },
    # Question 8 from AI Act
    {
        'law': 'ai_act',
        'question': "Quelles obligations les utilisateurs de systèmes d'IA à haut risque ont-ils selon l'AI Act?",
        'answer': "Les utilisateurs de systèmes d'IA à haut risque doivent exploiter les systèmes conformément aux instructions fournies par le fournisseur du système d'IA, surveiller le fonctionnement du système d'IA, et signaler rapidement tout incident grave ou dysfonctionnement au fournisseur et aux autorités compétentes. Les utilisateurs doivent également conserver les journaux générés par le système d'IA, s'assurer que la surveillance humaine est maintenue, et veiller à ce que le système d'IA soit utilisé uniquement à des fins prévues. De plus, les utilisateurs sont responsables de la mise en œuvre de mesures pour atténuer les risques pour les droits fondamentaux et la sécurité."
    },
    # Question 8 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA promeut-elle l'interopérabilité entre les services numériques?",
        'answer': "La DMA promeut l'interopérabilité en exigeant que les contrôleurs d'accès s'assurent que leurs services de plateforme essentiels peuvent interagir avec les services tiers. Cela inclut la mise à disposition des interfaces techniques et de la documentation nécessaires pour permettre l'interopérabilité. L'objectif est d'empêcher les contrôleurs d'accès de verrouiller les utilisateurs et les utilisateurs professionnels dans leurs plateformes et de permettre la concurrence en permettant aux nouveaux entrants et aux petits concurrents d'offrir des services complémentaires ou concurrents. L'interopérabilité est considérée comme une mesure clé pour promouvoir l'innovation et le choix des consommateurs sur les marchés numériques."
    },
    # Question 8 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA renforce-t-elle la protection des mineurs en ligne?",
        'answer': "La DSA inclut des dispositions spécifiques pour renforcer la protection des mineurs en ligne, reconnaissant que les enfants sont particulièrement vulnérables aux contenus et pratiques nuisibles. Les plateformes doivent mettre en œuvre des mesures pour s'assurer que leurs services sont sûrs pour les mineurs, y compris une modération de contenu adaptée à l'âge, des contrôles parentaux et des restrictions sur la publicité ciblée aux mineurs. La DSA exige également que les plateformes fournissent des informations claires et accessibles aux mineurs et à leurs parents sur les risques associés aux activités en ligne et sur la manière de se protéger. Ces mesures visent à créer un environnement en ligne plus sûr pour les enfants et à les autonomiser ainsi que leurs tuteurs pour qu'ils prennent des décisions éclairées."
    },
    # Question 9 from GDPR
    {
        'law': 'gdpr',
        'question': "Comment le RGPD traite-t-il la protection des données dès la conception et par défaut?",
        'answer': "Le RGPD exige que les responsables du traitement mettent en œuvre la protection des données dès la conception et par défaut. Cela signifie que les mesures de protection des données doivent être intégrées dans les activités de traitement dès le départ et que seules les données personnelles nécessaires à chaque finalité spécifique du traitement doivent être traitées. Le responsable du traitement doit prendre des mesures techniques et organisationnelles appropriées, telles que la pseudonymisation, pour garantir que, par défaut, les données personnelles ne soient pas rendues accessibles à un nombre indéterminé de personnes sans le consentement de l'individu."
    },
    # Question 9 from AI Act
    {
        'law': 'ai_act',
        'question': "Comment l'AI Act aborde-t-il l'utilisation des systèmes d'identification biométrique?",
        'answer': "L'AI Act impose des réglementations strictes sur l'utilisation des systèmes d'identification biométrique, en particulier ceux utilisés dans les espaces publics à des fins d'application de la loi. L'utilisation de systèmes d'identification biométrique à distance en temps réel dans des espaces accessibles au public est généralement interdite, des exceptions étant accordées sous des conditions spécifiques, telles que la prévention d'une attaque terroriste, la localisation d'un enfant disparu ou l'identification d'un suspect d'un crime grave. Même dans ces cas, l'utilisation doit être autorisée par des autorités judiciaires ou indépendantes et soumise à des garanties strictes pour protéger les droits fondamentaux."
    },
    # Question 9 from DMA
    {
        'law': 'dma',
        'question': "Quelles obligations la DMA impose-t-elle aux contrôleurs d'accès concernant l'accès et la portabilité des données?",
        'answer': "La DMA impose aux contrôleurs d'accès de fournir aux utilisateurs professionnels et aux utilisateurs finaux l'accès aux données générées par leurs interactions sur la plateforme. Cela inclut la fourniture de données dans un format structuré, couramment utilisé et lisible par machine pour faciliter la portabilité des données. Les contrôleurs d'accès doivent également permettre aux utilisateurs professionnels d'accéder aux données nécessaires au développement et à l'amélioration de leurs propres produits et services. Ces obligations visent à empêcher les contrôleurs d'accès d'utiliser leur contrôle sur les données pour étouffer la concurrence et l'innovation."
    },
    # Question 9 from DSA
    {
        'law': 'dsa',
        'question': "Quelles sont les obligations de transparence pour les plateformes en ligne concernant leurs algorithmes?",
        'answer': "La DSA impose des obligations de transparence aux plateformes en ligne pour fournir des informations claires et accessibles sur le fonctionnement de leurs algorithmes, en particulier ceux utilisés pour la modération de contenu, la recommandation et le classement. Les plateformes doivent expliquer les critères et la logique derrière leurs algorithmes, permettant aux utilisateurs de comprendre comment les décisions sont prises et comment le contenu leur est présenté. Les VLOPs ont des obligations supplémentaires de réaliser des audits algorithmiques et de permettre à des chercheurs indépendants d'évaluer l'impact de leurs algorithmes sur la société. Ces mesures de transparence visent à accroître la responsabilité et la confiance dans l'écosystème numérique."
    },
    # Question 10 from GDPR
    {
        'law': 'gdpr',
        'question': "Quel est le rôle du délégué à la protection des données (DPO) selon le RGPD?",
        'answer': "Le délégué à la protection des données (DPO) est responsable de la supervision des stratégies de protection des données et de la conformité aux exigences du RGPD. Le DPO doit être nommé par les autorités publiques et les organismes qui réalisent un suivi régulier et systématique des personnes concernées à grande échelle ou qui traitent des catégories particulières de données à grande échelle. Les responsabilités du DPO incluent le conseil à l'organisation sur les obligations du RGPD, la surveillance de la conformité, la formation du personnel, la réalisation d'audits, et le rôle de point de contact pour les autorités de contrôle et les personnes concernées."
    },
    # Question 10 from AI Act
    {
        'law': 'ai_act',
        'question': "Quelles sont les exigences pour les évaluations de conformité selon l'AI Act?",
        'answer': "Les systèmes d'IA à haut risque doivent faire l'objet d'une évaluation de conformité avant de pouvoir être mis sur le marché ou mis en service. Cette évaluation implique de vérifier si le système d'IA répond aux exigences énoncées dans l'AI Act, y compris la gestion des risques, la gouvernance des données, la transparence, la surveillance humaine et l'exactitude. L'évaluation peut être effectuée par le fournisseur ou par un organisme notifié, en fonction de la nature du système d'IA. L'évaluation de conformité doit être documentée, et le système d'IA doit porter un marquage CE indiquant la conformité au règlement."
    },
    # Question 10 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA aborde-t-elle la question des pratiques de couplage et de regroupement par les contrôleurs d'accès?",
        'answer': "La DMA interdit aux contrôleurs d'accès de se livrer à des pratiques de couplage et de regroupement qui obligent les utilisateurs à acheter ou utiliser des services supplémentaires comme condition d'accès au service de plateforme essentiel du contrôleur d'accès. Par exemple, un contrôleur d'accès ne peut pas exiger que les utilisateurs installent ou utilisent une application ou un service spécifique comme condition préalable à l'utilisation de leur plateforme. L'interdiction du couplage et du regroupement vise à empêcher les contrôleurs d'accès de tirer parti de leur pouvoir de marché pour étendre leur domination à d'autres marchés et à garantir que les utilisateurs ont la liberté de choisir les services qu'ils souhaitent utiliser."
    },
    # Question 10 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA aborde-t-elle la question de la désinformation et des fausses nouvelles sur les plateformes numériques?",
        'answer': "La DSA exige que les plateformes, en particulier les VLOPs, prennent des mesures proactives pour lutter contre la propagation de la désinformation et des fausses nouvelles. Cela inclut la mise en place de mécanismes pour détecter, évaluer et atténuer les risques associés à la désinformation, la collaboration avec des vérificateurs de faits indépendants, et la fourniture d'informations et de contextes précis aux utilisateurs. Les plateformes doivent également veiller à ce que leurs systèmes de modération de contenu et de recommandation n'amplifient ni ne favorisent la désinformation. La DSA promeut la transparence en exigeant que les plateformes rendent compte de leurs efforts pour lutter contre la désinformation et fournissent aux utilisateurs des outils pour identifier et signaler les fausses informations."
    },
    # Question 11 from GDPR
    {
        'law': 'gdpr',
        'question': "Quelles sont les implications du RGPD pour les activités de traitement des données transfrontalières?",
        'answer': "Le RGPD établit un cadre pour les activités de traitement des données transfrontalières afin de garantir que la protection des données soit cohérente dans toute l'UE. Les organisations qui traitent des données personnelles dans plusieurs États membres de l'UE doivent désigner une autorité de contrôle chef de file, qui agit comme point de contact unique pour superviser la conformité. Le RGPD facilite également la coopération entre les autorités de contrôle par le biais de mécanismes tels que le mécanisme de cohérence et le Comité européen de la protection des données (CEPD)."
    },
    # Question 11 from AI Act
    {
        'law': 'ai_act',
        'question': "Quel rôle jouent les autorités nationales de surveillance dans l'AI Act?",
        'answer': "Les autorités nationales de surveillance sont responsables de la mise en œuvre et de l'application de l'AI Act dans leurs juridictions respectives. Elles sont chargées de surveiller la conformité des systèmes d'IA aux exigences de la loi, de mener des inspections et des enquêtes, et de prendre des mesures d'application si nécessaire. Ces autorités jouent également un rôle clé dans la coordination avec les autres autorités nationales et la Commission européenne pour garantir une approche harmonisée de la réglementation de l'IA dans l'UE."
    },
    # Question 11 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA aborde-t-elle la question des fusions et acquisitions par les contrôleurs d'accès?",
        'answer': "La DMA exige que les contrôleurs d'accès informent la Commission européenne de toute fusion, acquisition ou concentration prévue impliquant d'autres fournisseurs de services de plateforme essentiels ou de services numériques. Cette obligation de notification permet à la Commission d'évaluer si la transaction proposée compromettrait les objectifs de la DMA, tels que le renforcement du pouvoir de marché du contrôleur d'accès ou la réduction de la concurrence sur les marchés numériques. Les dispositions de la DMA sur les fusions et acquisitions visent à empêcher les contrôleurs d'accès de consolider leur domination par le biais d'acquisitions stratégiques et à garantir que la concurrence reste robuste sur les marchés numériques."
    },
    # Question 11 from DSA
    {
        'law': 'dsa',
        'question': "Quel rôle jouent les signaleurs de confiance selon la DSA?",
        'answer': "La DSA reconnaît le rôle des signaleurs de confiance — des entités ayant une expertise dans l'identification des contenus illégaux — comme partenaires importants dans la modération de contenu. Les signaleurs de confiance bénéficient d'une priorité dans les mécanismes de notification et d'action, ce qui signifie que leurs signalements sont traités plus rapidement et avec plus de précision. Les plateformes doivent s'assurer que les signalements des signaleurs de confiance sont traités par des modérateurs expérimentés et qu'ils reçoivent des retours sur les actions entreprises. La désignation des signaleurs de confiance vise à améliorer l'efficacité et l'efficience de la modération de contenu, en particulier dans la lutte contre les contenus illégaux et les activités nuisibles en ligne."
    },
    # Question 12 from GDPR
    {
        'law': 'gdpr',
        'question': "Comment le RGPD gère-t-il les violations de données et quelles sont les obligations des responsables du traitement dans de tels cas?",
        'answer': "Selon le RGPD, les responsables du traitement doivent signaler les violations de données à l'autorité de contrôle compétente dans les 72 heures suivant la prise de connaissance de la violation, à moins que la violation ne soit pas susceptible d'engendrer un risque pour les droits et libertés des individus. Si la violation présente un risque élevé pour les individus concernés, le responsable du traitement doit également informer les personnes concernées sans retard injustifié. Le RGPD oblige les organisations à mettre en œuvre des mesures techniques et organisationnelles appropriées pour prévenir les violations de données et en atténuer l'impact."
    },
    # Question 12 from AI Act
    {
        'law': 'ai_act',
        'question': "Comment l'AI Act encourage-t-il l'innovation tout en garantissant la sécurité et la conformité?",
        'answer': "L'AI Act encourage l'innovation en fournissant des bacs à sable réglementaires, qui sont des environnements contrôlés où les développeurs d'IA peuvent tester leurs systèmes sous la supervision des autorités compétentes sans être immédiatement soumis aux exigences réglementaires complètes. Ces bacs à sable permettent l'expérimentation et le développement de solutions d'IA innovantes tout en garantissant que les normes de sécurité, d'éthique et de légalité sont maintenues. La loi promeut également l'adoption de codes de conduite volontaires pour les systèmes d'IA non à haut risque, permettant aux fournisseurs de démontrer leur engagement envers des pratiques éthiques en matière d'IA."
    },
    # Question 12 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA aborde-t-elle la question des schémas sombres et des pratiques de conception trompeuses par les contrôleurs d'accès?",
        'answer': "La DMA interdit aux contrôleurs d'accès d'utiliser des schémas sombres et des pratiques de conception trompeuses qui manipulent ou trompent les utilisateurs pour prendre des décisions qui ne sont pas dans leur meilleur intérêt. Cela inclut des pratiques telles que cacher des informations importantes, rendre difficile l'exercice des droits des utilisateurs, ou pousser les utilisateurs vers certains choix. La DMA oblige les contrôleurs d'accès à fournir des informations claires et accessibles aux utilisateurs et à concevoir leurs interfaces de manière à respecter l'autonomie et le choix des utilisateurs. Ces dispositions visent à protéger les consommateurs contre les pratiques manipulatrices et à garantir que les services numériques soient transparents et conviviaux."
    },
    # Question 12 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA promeut-elle la responsabilité des plateformes en ligne?",
        'answer': "La DSA promeut la responsabilité en imposant des exigences rigoureuses de rapport et de transparence aux plateformes en ligne. Les plateformes doivent publier des rapports de transparence réguliers détaillant leurs activités de modération de contenu, y compris le nombre d'actions de suppression, les raisons des suppressions et les résultats des appels des utilisateurs. Les VLOPs sont également tenues de se soumettre à des audits indépendants de leurs pratiques de modération de contenu et de gestion des risques. Ces audits visent à évaluer la conformité de la plateforme à la DSA et à identifier les domaines à améliorer. En promouvant la transparence et la responsabilité, la DSA vise à renforcer la confiance dans l'environnement numérique et à garantir que les plateformes agissent de manière responsable."
    },
    # Question 13 from GDPR
    {
        'law': 'gdpr',
        'question': "Quelles sont les restrictions sur le traitement des catégories particulières de données personnelles selon le RGPD?",
        'answer': "Le RGPD impose des règles plus strictes sur le traitement des catégories particulières de données personnelles, telles que les données révélant l'origine raciale ou ethnique, les opinions politiques, les croyances religieuses ou philosophiques, l'appartenance syndicale, les données génétiques, les données biométriques, les données de santé, et les données concernant la vie sexuelle ou l'orientation sexuelle d'une personne. Le traitement de ces données est interdit à moins que des conditions spécifiques ne soient remplies, telles que l'obtention du consentement explicite de la personne concernée, le respect des obligations légales dans le domaine de l'emploi et de la sécurité sociale, ou la protection des intérêts vitaux de la personne concernée."
    },
    # Question 13 from AI Act
    {
        'law': 'ai_act',
        'question': "Comment l'AI Act aborde-t-il la transparence des systèmes d'IA?",
        'answer': "L'AI Act exige que les systèmes d'IA, en particulier ceux à haut risque, soient conçus et développés en tenant compte de la transparence. Cela inclut la fourniture d'informations claires et accessibles aux utilisateurs sur l'objectif, les capacités, les limites et le fonctionnement du système d'IA. Les utilisateurs doivent être informés lorsqu'ils interagissent avec un système d'IA, en particulier dans les cas où l'IA est utilisée pour prendre des décisions ayant des impacts importants sur les individus. Les exigences de transparence visent à garantir que les utilisateurs et les personnes concernées comprennent comment et pourquoi les décisions sont prises par les systèmes d'IA."
    },
    # Question 13 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA promeut-elle la transparence dans la publicité numérique?",
        'answer': "La DMA promeut la transparence dans la publicité numérique en obligeant les contrôleurs d'accès à fournir aux annonceurs et éditeurs l'accès aux données relatives à leurs campagnes publicitaires, y compris des informations sur les prix, les performances et les critères de ciblage. Les contrôleurs d'accès doivent également s'assurer que leurs services publicitaires sont proposés à des conditions équitables, raisonnables et non discriminatoires, et il leur est interdit d'utiliser des données non publiques pour obtenir un avantage injuste sur le marché publicitaire. Ces dispositions visent à promouvoir la concurrence et la transparence dans la publicité numérique, en garantissant que les annonceurs et les éditeurs disposent des informations nécessaires pour prendre des décisions éclairées."
    },
    # Question 13 from DSA
    {
        'law': 'dsa',
        'question': "Quelles sont les sanctions pour non-conformité à la DSA?",
        'answer': "La DSA prévoit des sanctions substantielles pour non-conformité, y compris des amendes pouvant atteindre 6 % du chiffre d'affaires annuel mondial total de la plateforme. En cas de non-conformité répétée ou grave, la DSA permet des mesures supplémentaires, telles que la suspension temporaire des services de la plateforme ou d'autres actions correctives. L'application de la DSA est supervisée par les autorités nationales de régulation, qui ont le pouvoir d'enquêter et de sanctionner les plateformes qui violent la réglementation. Ces sanctions sont conçues pour garantir que les plateformes prennent leurs obligations au sérieux et que les dispositions de la DSA sont effectivement mises en œuvre."
    },
    # Question 14 from GDPR
    {
        'law': 'gdpr',
        'question': "Comment le RGPD régule-t-il la prise de décision automatisée et le profilage?",
        'answer': "Le RGPD impose des restrictions sur la prise de décision automatisée, y compris le profilage, lorsque des décisions sont prises uniquement sur la base d'un traitement automatisé et affectent de manière significative les individus. Un tel traitement n'est autorisé que dans des situations spécifiques, telles que lorsqu'il est nécessaire à la conclusion ou à l'exécution d'un contrat, autorisé par le droit de l'Union ou des États membres, ou basé sur le consentement explicite de la personne concernée. Les organisations doivent s'assurer que les individus sont informés de l'existence de la prise de décision automatisée, de la logique impliquée et des conséquences potentielles. Les personnes concernées ont le droit de contester les décisions automatisées et de demander une intervention humaine."
    },
    # Question 14 from AI Act
    {
        'law': 'ai_act',
        'question': "Quelles sont les obligations liées à la qualité des données selon l'AI Act?",
        'answer': "L'AI Act exige que les systèmes d'IA à haut risque soient formés, testés et validés à l'aide de jeux de données de haute qualité qui sont pertinents, représentatifs, exempts d'erreurs et complets. Les données doivent être soigneusement sélectionnées pour éviter les biais qui pourraient conduire à des résultats discriminatoires. Les fournisseurs doivent s'assurer que le cadre de gouvernance des données comprend des mesures pour évaluer et atténuer les risques liés à la qualité des données, comme l'utilisation de jeux de données diversifiés et représentatifs, la validation de l'exactitude et de la fiabilité des données, et la mise à jour régulière des jeux de données pour refléter les changements au fil du temps."
    },
    # Question 14 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA aborde-t-elle la question de l'accès aux services de plateforme essentiels par les utilisateurs finaux?",
        'answer': "La DMA garantit que les utilisateurs finaux ont accès aux services de plateforme essentiels à des conditions équitables et non discriminatoires. Les contrôleurs d'accès ne peuvent pas restreindre ou dégrader la qualité d'accès à leurs services ni se livrer à des pratiques qui limitent le choix des utilisateurs, telles que forcer les utilisateurs à installer certaines applications ou à utiliser des services spécifiques. La DMA promeut également la portabilité des données, permettant aux utilisateurs finaux de transférer leurs données vers d'autres services et de profiter d'offres concurrentielles. Ces dispositions visent à améliorer le choix et le contrôle des utilisateurs sur les services numériques qu'ils utilisent."
    },
    # Question 14 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA soutient-elle les droits des consommateurs dans le marché numérique?",
        'answer': "La DSA renforce les droits des consommateurs en veillant à ce que les plateformes en ligne fournissent des informations claires et accessibles sur les biens, services et contenus disponibles sur leurs plateformes. Cela inclut l'obligation pour les plateformes de divulguer des informations sur l'identité des vendeurs, les conditions des transactions et la nature des biens et services proposés. Les consommateurs doivent également être informés de leurs droits, y compris le droit de se retirer d'une transaction, le droit à un remboursement et le droit d'accéder à des mécanismes de résolution des litiges efficaces. Les dispositions de protection des consommateurs de la DSA sont conçues pour créer un marché numérique sûr et transparent."
    },
    # Question 15 from GDPR
    {
        'law': 'gdpr',
        'question': "Quelles sanctions et mesures d'application sont prévues par le RGPD?",
        'answer': "Le RGPD prévoit des sanctions et des mesures d'application substantielles pour assurer la conformité. Les autorités de contrôle ont le pouvoir d'imposer des amendes administratives pouvant atteindre 20 millions d'euros ou 4% du chiffre d'affaires annuel mondial total de l'exercice précédent, selon le montant le plus élevé, pour les violations les plus graves. Les sanctions sont déterminées en fonction de facteurs tels que la nature, la gravité et la durée de l'infraction, le caractère intentionnel ou négligent de l'infraction, et les mesures prises par l'organisation pour atténuer les dommages."
    },
    # Question 15 from AI Act
    {
        'law': 'ai_act',
        'question': "Comment l'AI Act réglemente-t-il l'utilisation de l'IA dans l'application de la loi et la sécurité publique?",
        'answer': "L'AI Act impose des réglementations strictes sur l'utilisation des systèmes d'IA dans l'application de la loi et la sécurité publique, en particulier ceux utilisés pour la police prédictive, l'identification biométrique et la surveillance. Ces systèmes sont considérés comme à haut risque et font l'objet d'un examen rigoureux pour s'assurer qu'ils ne portent pas atteinte aux droits fondamentaux, tels que la vie privée et la non-discrimination. Les agences chargées de l'application de la loi doivent réaliser une évaluation détaillée des risques et mettre en œuvre des garanties pour s'assurer que l'utilisation des systèmes d'IA est nécessaire, proportionnée et respectueuse des droits de l'homme."
    },
    # Question 15 from DMA
    {
        'law': 'dma',
        'question': "Quel rôle la Commission européenne joue-t-elle dans l'application de la DMA?",
        'answer': "La Commission européenne est responsable de l'application de la DMA, y compris la surveillance de la conformité, la conduite d'enquêtes et l'imposition de sanctions pour non-conformité. La Commission a le pouvoir d'imposer des amendes, des paiements de pénalités périodiques et des mesures structurelles aux contrôleurs d'accès qui violent les obligations et interdictions de la DMA. La Commission a également le pouvoir de lancer des enquêtes de marché pour évaluer si de nouveaux services doivent être désignés comme services de plateforme essentiels ou si des obligations supplémentaires doivent être imposées aux contrôleurs d'accès. L'application de la DMA est conçue pour être robuste et efficace, garantissant que les contrôleurs d'accès opèrent de manière à promouvoir la concurrence et l'innovation sur les marchés numériques."
    },
    # Question 15 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA gère-t-elle la question du harcèlement et des abus en ligne?",
        'answer': "La DSA oblige les plateformes à mettre en œuvre des mesures pour lutter contre le harcèlement et les abus en ligne, y compris en fournissant aux utilisateurs des outils pour signaler et bloquer les contenus et comportements abusifs. Les plateformes doivent agir rapidement pour supprimer ou désactiver l'accès aux contenus qui constituent du harcèlement ou des abus, et elles doivent apporter un soutien aux victimes. La DSA encourage également les plateformes à collaborer avec les forces de l'ordre et les organisations de la société civile pour traiter le harcèlement en ligne et développer des bonnes pratiques pour créer un environnement en ligne sûr. Ces mesures visent à protéger les utilisateurs contre les préjudices et à promouvoir un espace numérique respectueux et inclusif."
    },
    # Question 16 from GDPR
    {
        'law': 'gdpr',
        'question': "Quel est le rôle du Comité européen de la protection des données (CEPD) selon le RGPD?",
        'answer': "Le Comité européen de la protection des données (CEPD) est un organisme indépendant créé par le RGPD pour garantir l'application cohérente des règles de protection des données dans l'UE. Le CEPD est composé de représentants des autorités nationales de protection des données et du Contrôleur européen de la protection des données (CEPD). Ses responsabilités incluent la publication de lignes directrices, de recommandations et de bonnes pratiques sur l'interprétation et l'application du RGPD, la résolution des litiges entre autorités de contrôle, et la fourniture de conseils à la Commission européenne sur les questions de protection des données."
    },
    # Question 16 from AI Act
    {
        'law': 'ai_act',
        'question': "Comment l'AI Act aborde-t-il la question des biais et de la discrimination dans les systèmes d'IA?",
        'answer': "L'AI Act exige que les systèmes d'IA, en particulier ceux à haut risque, soient conçus et développés de manière à prévenir, identifier et atténuer les biais qui pourraient conduire à des résultats discriminatoires. Les fournisseurs doivent prendre des mesures pour s'assurer que les systèmes d'IA ne produisent pas de résultats qui désavantagent injustement des individus ou des groupes sur la base de caractéristiques protégées telles que la race, le sexe ou la religion. Cela inclut l'utilisation de jeux de données diversifiés, la réalisation d'audits de biais, et la mise en œuvre de mesures correctives pour traiter tout biais identifié. La loi met également l'accent sur l'importance de la surveillance humaine pour prévenir et traiter les biais."
    },
    # Question 16 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA aborde-t-elle la question des schémas sombres et des pratiques de conception trompeuses par les contrôleurs d'accès?",
        'answer': "La DMA interdit aux contrôleurs d'accès d'utiliser des schémas sombres et des pratiques de conception trompeuses qui manipulent ou trompent les utilisateurs pour prendre des décisions qui ne sont pas dans leur meilleur intérêt. Cela inclut des pratiques telles que cacher des informations importantes, rendre difficile l'exercice des droits des utilisateurs, ou pousser les utilisateurs vers certains choix. La DMA oblige les contrôleurs d'accès à fournir des informations claires et accessibles aux utilisateurs et à concevoir leurs interfaces de manière à respecter l'autonomie et le choix des utilisateurs. Ces dispositions visent à protéger les consommateurs contre les pratiques manipulatrices et à garantir que les services numériques soient transparents et conviviaux."
    },
    # Question 16 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA garantit-elle que les utilisateurs ont le contrôle sur leurs données et leur vie privée?",
        'answer': "La DSA renforce le contrôle des utilisateurs sur leurs données et leur vie privée en exigeant que les plateformes fournissent des informations claires et accessibles sur la manière dont les données des utilisateurs sont collectées, traitées et utilisées. Les utilisateurs doivent être informés de leurs droits d'accès, de rectification et de suppression de leurs données, ainsi que de leur droit de s'opposer au traitement des données. La DSA oblige également les plateformes à mettre en œuvre les principes de protection de la vie privée dès la conception et par défaut, garantissant que la vie privée des utilisateurs est protégée dès le départ. De plus, les plateformes doivent fournir aux utilisateurs des outils pour gérer leurs paramètres de confidentialité et contrôler l'utilisation de leurs données pour la publicité ciblée."
    },
    # Question 17 from GDPR
    {
        'law': 'gdpr',
        'question': "Comment le RGPD aborde-t-il la question du consentement dans le traitement des données?",
        'answer': "Selon le RGPD, le consentement doit être donné librement, spécifiquement, informé et sans ambiguïté. Les organisations doivent s'assurer que le consentement est obtenu par une action affirmative claire, telle que cocher une case sur un site web, et qu'il est distinct des autres sujets. La personne concernée doit être informée de son droit de retirer son consentement à tout moment, et le retrait doit être aussi simple que de donner son consentement. De plus, pour les enfants de moins de 16 ans, le consentement parental est requis pour le traitement de leurs données."
    },
    # Question 17 from AI Act
    {
        'law': 'ai_act',
        'question': "Quel est le rôle du Conseil européen de l'intelligence artificielle (CEIA) dans l'AI Act?",
        'answer': "Le Conseil européen de l'intelligence artificielle (CEIA) est établi dans l'AI Act pour faciliter la coopération et la coordination entre les autorités nationales de surveillance et la Commission européenne. Le CEIA est responsable de l'émission de lignes directrices, de recommandations et de bonnes pratiques sur la mise en œuvre de l'AI Act, de fournir des conseils à la Commission européenne sur les questions liées à l'IA, et de promouvoir l'application harmonisée de la loi dans l'UE. Le CEIA joue également un rôle dans la résolution des litiges entre les autorités nationales et assure la cohérence dans l'interprétation et l'application de l'AI Act."
    },
    # Question 17 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA promeut-elle l'innovation et la concurrence sur les marchés numériques?",
        'answer': "La DMA promeut l'innovation et la concurrence en empêchant les contrôleurs d'accès de se livrer à des pratiques qui étouffent la concurrence, telles que l'auto-préférence, le couplage et le regroupement. En veillant à ce que les contrôleurs d'accès opèrent à des conditions équitables, raisonnables et non discriminatoires, la DMA crée des opportunités pour que les nouveaux entrants et les petits concurrents puissent concurrencer à armes égales. La DMA promeut également l'interopérabilité et la portabilité des données, permettant aux entreprises de développer des services innovants pouvant interagir avec la plateforme du contrôleur d'accès. Ces mesures visent à favoriser un marché numérique dynamique et concurrentiel qui profite aux consommateurs et aux entreprises."
    },
    # Question 17 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA aborde-t-elle la question de la transparence et de la responsabilité algorithmiques?",
        'answer': "La DSA exige que les plateformes, en particulier les VLOPs, fournissent des informations transparentes sur le fonctionnement de leurs algorithmes, y compris les critères utilisés pour la recommandation, le classement et la suppression de contenu. Les plateformes doivent expliquer la logique derrière leurs algorithmes et fournir aux utilisateurs des options pour contrôler la manière dont les algorithmes affectent leur expérience en ligne. La DSA oblige également les plateformes à réaliser des audits réguliers de leurs algorithmes pour évaluer leur impact sur les utilisateurs et la société. Ces audits doivent être menés par des tiers indépendants et doivent évaluer si les algorithmes sont équitables, non discriminatoires et alignés sur les droits fondamentaux."
    },
    # Question 18 from GDPR
    {
        'law': 'gdpr',
        'question': "Quelle est l'approche du RGPD concernant les transferts internationaux de données?",
        'answer': "Le RGPD autorise les transferts internationaux de données uniquement si le pays tiers, le territoire ou l'organisation internationale garantit un niveau adéquat de protection des données, tel que déterminé par la Commission européenne. En l'absence de décision d'adéquation, les transferts sont permis sous des garanties appropriées, telles que des règles d'entreprise contraignantes ou des clauses contractuelles types. Dans des circonstances spécifiques, des dérogations pour des situations particulières, telles que le consentement explicite de la personne concernée, peuvent permettre des transferts. Le RGPD vise à garantir que les données personnelles transférées en dehors de l'UE bénéficient du même niveau de protection que dans l'UE."
    },
    # Question 18 from AI Act
    {
        'law': 'ai_act',
        'question': "Comment l'AI Act impacte-t-il l'utilisation de l'IA dans le domaine de la santé?",
        'answer': "L'AI Act reconnaît les avantages potentiels de l'IA dans le domaine de la santé, tels que l'amélioration du diagnostic, du traitement et des résultats pour les patients. Cependant, il reconnaît également les risques associés à l'utilisation de l'IA dans ce secteur sensible. Les systèmes d'IA utilisés dans le domaine de la santé, en particulier ceux qui impliquent la prise de décisions ou fournissent des recommandations aux professionnels de santé, sont classés comme à haut risque et sont soumis à des exigences strictes. Cela inclut la garantie de l'exactitude et de la fiabilité des systèmes d'IA, le maintien de la surveillance humaine, et la protection des données des patients. La loi met également l'accent sur l'importance de la transparence et du consentement éclairé dans l'utilisation de l'IA en santé."
    },
    {
        'law': 'gdpr',
        'question': "Comment le RGPD impacte-t-il les petites et moyennes entreprises (PME)?",
        'answer': "Le RGPD reconnaît les défis auxquels sont confrontées les petites et moyennes entreprises (PME) et prévoit certaines exemptions et dérogations pour alléger leur charge de conformité. Par exemple, les PME de moins de 250 employés ne sont pas tenues de tenir des registres des activités de traitement, sauf si le traitement est régulier, présente un risque pour les droits et libertés des personnes concernées, ou implique des catégories particulières de données. Cependant, les PME doivent encore se conformer aux autres exigences du RGPD, telles que la notification des violations de données, les droits des personnes concernées et la nomination d'un DPO si nécessaire."
    },
    # Question 19 from AI Act
    {
        'law': 'ai_act',
        'question': "Comment l'AI Act aborde-t-il la question de la littératie en IA et de la sensibilisation du public?",
        'answer': "L'AI Act encourage les initiatives pour promouvoir la littératie en IA et la sensibilisation du public, reconnaissant que des citoyens informés et éduqués sont essentiels pour une adoption responsable des technologies d'IA. La loi appelle au développement de programmes éducatifs et de ressources pour aider les individus à comprendre les capacités, les limites et les risques associés à l'IA. Elle encourage également les consultations publiques et la participation des parties prenantes pour garantir que les perspectives de divers groupes, y compris la société civile, soient prises en compte dans le développement et le déploiement des systèmes d'IA."
    },
    # Question 20 from GDPR
    {
        'law': 'gdpr',
        'question': "Quelles mesures le RGPD recommande-t-il pour garantir la sécurité des données?",
        'answer': "Le RGPD oblige les organisations à mettre en œuvre des mesures techniques et organisationnelles appropriées pour garantir un niveau de sécurité adapté au risque. Cela inclut des mesures telles que la pseudonymisation et le chiffrement des données personnelles, la garantie de la confidentialité, de l'intégrité, de la disponibilité et de la résilience continues des systèmes et services de traitement, et des tests, évaluations et évaluations réguliers de l'efficacité des mesures de sécurité. Le RGPD insiste également sur la nécessité d'évaluations d'impact sur la protection des données (DPIA) lorsque les opérations de traitement sont susceptibles de présenter un risque élevé pour les droits et libertés des personnes physiques."
    },
    # Question 20 from AI Act
    {
        'law': 'ai_act',
        'question': "Quelles mesures l'AI Act inclut-il pour soutenir le développement éthique de l'IA?",
        'answer': "L'AI Act soutient le développement éthique de l'IA en encourageant l'adoption de codes de conduite volontaires, en favorisant la recherche sur l'IA éthique, et en promouvant le développement de systèmes d'IA qui s'alignent sur les valeurs européennes et les droits fondamentaux. La loi insiste sur l'importance d'une IA centrée sur l'humain, où les systèmes d'IA sont conçus pour améliorer les capacités humaines et le bien-être tout en respectant la dignité et l'autonomie humaines. Elle soutient également la création de bacs à sable réglementaires pour permettre aux développeurs d'expérimenter des solutions d'IA innovantes dans un environnement contrôlé, en s'assurant que les considérations éthiques sont intégrées dans la conception et le déploiement des technologies d'IA."
    },
    # Question 20 from DMA
    {
        'law': 'dma',
        'question': "Comment la DMA promeut-elle la transparence dans la publicité numérique?",
        'answer': "La DMA promeut la transparence dans la publicité numérique en obligeant les contrôleurs d'accès à fournir aux annonceurs et éditeurs l'accès aux données relatives à leurs campagnes publicitaires, y compris des informations sur les prix, les performances et les critères de ciblage. Les contrôleurs d'accès doivent également s'assurer que leurs services publicitaires sont proposés à des conditions équitables, raisonnables et non discriminatoires, et il leur est interdit d'utiliser des données non publiques pour obtenir un avantage injuste sur le marché publicitaire. Ces dispositions visent à promouvoir la concurrence et la transparence dans la publicité numérique, en garantissant que les annonceurs et les éditeurs disposent des informations nécessaires pour prendre des décisions éclairées."
    },
    # Question 20 from DSA
    {
        'law': 'dsa',
        'question': "Comment la DSA promeut-elle le développement de codes de conduite pour les plateformes en ligne?",
        'answer': "La DSA encourage le développement de codes de conduite pour les plateformes en ligne pour traiter des questions spécifiques telles que la modération de contenu, la transparence algorithmique et la protection des mineurs. Ces codes de conduite sont élaborés en collaboration avec les parties prenantes de l'industrie, les organisations de la société civile et les autorités de régulation. La DSA promeut l'adoption de ces mesures volontaires pour s'assurer que les plateformes opèrent de manière responsable et éthique. Les codes de conduite fournissent un cadre pour les meilleures pratiques et aident les plateformes à aligner leurs opérations sur les objectifs de la DSA, tout en permettant flexibilité et innovation."
    }
    ]

laws_info = {
    'gdpr': {
        'file_path': '/kaggle/input/datasetssss/french_gdpr.html',
        'collection_name': 'chunk-embeddings-index',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'gdpr']
    },
    'ai_act': {
        'file_path': '/kaggle/input/datasetssss/french_AI_act.html',
        'collection_name': 'chunk-embeddings-index',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'ai_act']
    },
    'dma': {
        'file_path': '/kaggle/input/datasetssss/french_dma.html',
        'collection_name': 'DMA_Chunk',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'dma']
    },
    'dsa': {
        'file_path': '/kaggle/input/datasetssss/french_dsa.html',
        'collection_name': 'DSA_Chunk',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'dsa']
    },
}

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
print(f"Summarizer device: {summarizer.device}")
print(f"Cosine model device: {cosine_model.device}")
print(f"Semantic model device: {semantic_model.device}")
model.device

In [ ]:
from collections import defaultdict
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum",device=0 if torch.cuda.is_available() else -1)
cosine_model = SentenceTransformer('microsoft/deberta-v3-base',device=device) 


def embed_query(query, model_name):
    return cosine_model.encode(query, convert_to_tensor=True)
    
def summarize_text_huggingface_with_retry(text, max_length=350, min_length=100, max_retries=3):
    for attempt in range(max_retries):
        try:
            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
            return summary[0]['summary_text']
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                print("Max retries reached. Returning None.")
                return None

def calculate_cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    
for law, law_info in laws_info.items():
    print(f"\nProcessing law: {law.upper()}")
    
    for qa in law_info['questions_answers']:
        question = qa['question']
        reference_answer = qa['answer']
        
        print(f"Querying for question: {question}")

        query_embedding = embed_query(question, model_name)

        results = query_pinecone_db(query_embedding, index, namespace=law, top_k=1)

        if results and 'matches' in results and results['matches']:
            retrieved_chunk_id = results['matches'][0]['id']
            
            if law in chunks_dict:
                retrieved_chunk_index = int(retrieved_chunk_id.split('_')[-1])
                retrieved_chunk = chunks_dict[law][retrieved_chunk_index]

                print(f"Retrieved chunk {retrieved_chunk_index} from {law.upper()}:")
                print(retrieved_chunk)

                summary = summarize_text_huggingface_with_retry(retrieved_chunk)
                print(summary)
                qa['summary'] = summary
                if summary:
                    answer_embedding = cosine_model.encode(reference_answer, convert_to_tensor=False)
                    summary_embedding = cosine_model.encode(summary, convert_to_tensor=False)
                    qa['answer_embedding'] = answer_embedding
                    qa['summary_embedding'] = summary_embedding
                    
                    cosine_similarity = calculate_cosine_similarity(answer_embedding, summary_embedding)
                    qa['cosine_similarity'] = cosine_similarity
                    
                    semantic_similarity = util.pytorch_cos_sim(
                        semantic_model.encode(reference_answer, convert_to_tensor=True),
                        semantic_model.encode(summary, convert_to_tensor=True)
                    ).item()
                    qa['semantic_similarity'] = semantic_similarity
                    
                    print(f"Question: {question}")
                    print(f"Summary: {summary}")
                    print(f"Cosine Similarity: {cosine_similarity:.4f}")
                    print(f"Semantic Similarity: {semantic_similarity:.4f}")
                else:
                    print(f"No summary generated for question: {question}")

            else:
                print(f"Error: '{law}' not found in chunks_dict.")
        else:
            print(f"No results found for {law.upper()} for the question: {question}")

averages = {}
for law, info in laws_info.items():
    total_cosine = 0
    total_semantic = 0
    count = 0

    for qa in info['questions_answers']:
        if 'cosine_similarity' in qa and 'semantic_similarity' in qa:
            total_cosine += qa['cosine_similarity']
            total_semantic += qa['semantic_similarity']
            count += 1
    
    if count > 0:
        avg_cosine = total_cosine / count
        avg_semantic = total_semantic / count
        averages[law] = {
            'average_cosine_similarity': avg_cosine,
            'average_semantic_similarity': avg_semantic
        }
        print(f"\nAverage Cosine Similarity for {law.upper()}: {avg_cosine:.4f}")
        print(f"Average Semantic Similarity for {law.upper()}: {avg_semantic:.4f}")
    else:
        print(f"\nNo valid similarity scores for {law.upper()}.")

In [ ]:
import time

def measure_and_average_query_latency(laws_info, model, tokenizer, top_k=1):
    latency_results = {law: [] for law in laws_info.keys()} 
    all_latencies = []

    for law, info in laws_info.items():
        print(f"\nMeasuring query latency for {law.upper()} namespace:")

        if not info.get('questions_answers', []):
            print(f"No questions found for law: {law}")
            continue

        for qa in info['questions_answers']:
            query = qa.get('question', None)
            if not query:
                print(f"Skipping missing query for law: {law}")
                continue

            try:
                query_embedding = get_embedding(query, model, tokenizer)
                print(f"Generated embedding for query: {query}")
            except Exception as e:
                print(f"Error generating embedding for query: {query}. Error: {e}")
                continue

            start_time = time.time()

            try:
                results = index.query(
                    vector=query_embedding.tolist(), 
                    top_k=top_k,
                    namespace=law
                )
            except Exception as e:
                print(f"Error querying Pinecone for {query}: {e}")
                continue

            end_time = time.time()

            latency = end_time - start_time
            latency_results[law].append(latency)
            all_latencies.append(latency)

            print(f"Query: {query}")
            print(f"Latency: {latency:.4f} seconds")
            print("----\n")
    
    for law in latency_results:
        print(f"Latencies for {law}: {latency_results[law]}")
        if latency_results[law]:  
            avg_latency = sum(latency_results[law]) / len(latency_results[law])
            print(f"{law.upper()} Average Query Latency: {avg_latency:.4f} seconds")
        else:
            print(f"{law.upper()} has no recorded latencies.")

    if all_latencies:
        overall_avg_latency = sum(all_latencies) / len(all_latencies)
        print(f"\nOverall Average Query Latency: {overall_avg_latency:.4f} seconds")
    else:
        print("No latencies recorded across all laws.")

def get_embedding(text, model, tokenizer, max_length=512):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_length
    )
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()

from transformers import AutoTokenizer, AutoModel

embedding_model = AutoModel.from_pretrained("camembert-base")
tokenizer = AutoTokenizer.from_pretrained("camembert-base")

measure_and_average_query_latency(laws_info, embedding_model, tokenizer, top_k=5)

# Load the other 80 questions, to retrieve the most relevant chunks, retrieve, making sums and compare them with the answers

In [ ]:
import pandas as pd

def consolidate_french_csv(file_path, output_path):
    try:
        print(f"Processing file: {file_path}")

        df = pd.read_csv(file_path, encoding='utf-8', header=None)

        consolidated_column = df.apply(lambda row: " ".join(row.dropna().astype(str)).strip(), axis=1)

        consolidated_column = consolidated_column[consolidated_column != ""]

        output_df = pd.DataFrame({df.columns[0]: consolidated_column})

        print(f"Consolidated DataFrame:\n{output_df.head()}")

        output_df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"Processed file saved to: {output_path}")

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

french_csv_path = '/kaggle/input/datasetssss/questions_answers  gdpr fr.csv'
output_path = '/kaggle/working/consolidated_french_gdpr.csv'

consolidate_french_csv(french_csv_path, output_path)


In [ ]:
def process_csv(file_path, output_path):
    try:
        print(f"\nProcessing file: {file_path}")

        df = pd.read_csv(file_path, encoding='utf-8', header=None)

        main_column = df.apply(lambda row: " ".join(row.dropna().astype(str)).strip(), axis=1)

        main_column = main_column[main_column != ""]

        print(f"First 5 Rows of Consolidated Data:\n{main_column.head()}")

        questions = []
        answers = []
        current_question = None

        for text in main_column:
            text = str(text).strip()  # Clean the text

            if text.startswith("Question :"):
                current_question = text.replace("Question :", "").strip()
            elif text.startswith("Réponse :") and current_question:
                answer = text.replace("Réponse :", "").strip()
                questions.append(current_question)
                answers.append(answer)
                current_question = None  
            else:
                continue  

        qa_df = pd.DataFrame({'Question': questions, 'Réponse': answers})

        print(f"Extracted DataFrame:\n{qa_df.head()}")

        qa_df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"Processed file saved to: {output_path}")

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

file_paths = {
    'gdpr': ('/kaggle/input/datasetssss/questions_answers  gdpr fr.csv', '/kaggle/working/consolidated_french_gdpr.csv'),
    'ai_act': ('/kaggle/input/datasetssss/questions_answers_ai_test fr.csv', '/kaggle/working/consolidated_french_ai_act.csv'),
    'dma': ('/kaggle/input/datasetssss/questions_answers_digital_marketing fr.csv', '/kaggle/working/consolidated_french_dma.csv'),
    'dsa': ('/kaggle/input/datasetssss/questions_answers_digital_services fr.csv', '/kaggle/working/consolidated_french_dsa.csv'),
}

for law, paths in file_paths.items():
    input_path, output_path = paths
    process_csv(input_path, output_path)

In [ ]:
import pandas as pd
import pinecone
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch


summarizer = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum",device=-1)
cosine_model = SentenceTransformer('microsoft/deberta-v3-base', device='cpu')
semantic_model = SentenceTransformer('google/mt5-base', device='cpu')

def safe_truncate(text, max_tokens=1024):
    tokens = tokenizer.tokenize(text)
    if len(tokens) > max_tokens:
        print(f"Truncating text from {len(tokens)} to {max_tokens} tokens.")
        tokens = tokens[:max_tokens]
    return tokenizer.convert_tokens_to_string(tokens)

def store_embeddings_in_pinecone(chunks, embeddings, namespace):
    vectors = [
        (f"{namespace}_id_{i}", embedding.tolist(), {"text": chunk})
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings))
    ]
    index.upsert(vectors=vectors, namespace=namespace)

def embed_and_query(query_text, model_norm, namespace, top_k=1):
    try:
        query_embedding = model_norm.encode([query_text])[0]
        print(f"Query embedding created for: {query_text}")

        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            namespace=namespace
        )
        return results
    except Exception as e:
        print(f"Error querying Pinecone for namespace '{namespace}': {e}")
        return None
        
def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

def calculate_semantic_similarity(text1, text2):
    embedding1 = semantic_model.encode(text1)
    embedding2 = semantic_model.encode(text2)
    return cosine_similarity([embedding1], [embedding2])[0][0]

def summarize_and_compare_all_laws(qa_dataframes, model_norm, namespaces):
    similarities = {law: {'cosine': [], 'semantic': []} for law in namespaces.keys()}

    for law, df in qa_dataframes.items():
        print(f"\nProcessing summaries for {law.upper()} law:")
        question_col = 'Question'
        answer_col = 'Réponse'

        for _, row in df.iterrows():
            question_text = row[question_col]
            answer_text = row[answer_col]
        
            print(f"Querying for: {question_text}")
            try:
                results = embed_and_query(question_text, model_norm, namespaces[law], top_k=1)
                if results and 'matches' in results and results['matches']:
                    retrieved_chunk_id = results['matches'][0]['id']
                if law in chunks_dict:
                    retrieved_chunk_index = int(retrieved_chunk_id.split('_')[-1])
                    retrieved_chunk = chunks_dict[law][retrieved_chunk_index]
    
                    print(f"Retrieved chunk {retrieved_chunk_index} from {law.upper()}:")
                    print(retrieved_chunk)
            
            except Exception as e:
                print(f"Error processing question '{question_text}': {e}")
                continue

        
            truncated_text = safe_truncate(retrieved_chunk, max_tokens=1024)
            try:
                summary = summarizer(truncated_text, max_length=350, min_length=100, do_sample=False)[0]['summary_text']
            except Exception as e:
                print(f"Failed to summarize chunk for question: {question_text}. Error: {e}")
                continue
        
            if summary:
                print(f"Original answer: {answer_text}")
                print(f"Generated summary: {summary}\n")
        
                try:
                    answer_embedding = model_norm.encode([answer_text])[0]
                    summary_embedding = model_norm.encode([summary])[0]
        
                    cosine_sim = calculate_cosine_similarity(answer_embedding, summary_embedding)
                    semantic_sim = calculate_semantic_similarity(answer_text, summary)
        
                    similarities[law]['cosine'].append(cosine_sim)
                    similarities[law]['semantic'].append(semantic_sim)
        
                    print(f"Cosine similarity: {cosine_sim:.4f}")
                    print(f"Semantic similarity: {semantic_sim:.4f}\n")
                except Exception as e:
                    print(f"Error calculating similarities for question: {question_text}. Error: {e}")
            else:
                print(f"Failed to summarize for: {question_text}")

    print("Calculated Averages:")
    for law in similarities:
        cosine_scores = similarities[law]['cosine']
        semantic_scores = similarities[law]['semantic']
        if cosine_scores:
            avg_cosine = sum(cosine_scores) / len(cosine_scores)
            print(f"{law.upper()} Average Cosine Similarity: {avg_cosine:.4f}")
        if semantic_scores:
            avg_semantic = sum(semantic_scores) / len(semantic_scores)
            print(f"{law.upper()} Average Semantic Similarity: {avg_semantic:.4f}")
    return similarities

qa_dataframes = {
    'gdpr': pd.read_csv('/kaggle/working/consolidated_french_gdpr.csv'),
    'ai_act': pd.read_csv('/kaggle/working/consolidated_french_ai_act.csv'),
    'dma': pd.read_csv('/kaggle/working/consolidated_french_dma.csv'),
    'dsa': pd.read_csv('/kaggle/working/consolidated_french_dsa.csv'),
}

for law, df in qa_dataframes.items():
    chunks = df['Réponse'].tolist()
    embeddings = cosine_model.encode(chunks)
    store_embeddings_in_pinecone(chunks, embeddings, namespace=law)

namespaces = {law: law for law in qa_dataframes.keys()}
summarize_and_compare_all_laws(qa_dataframes, cosine_model, namespaces)